In [1]:
from pyspark.ml.feature import VectorAssembler
df=spark.sql("select * from user_erin.creditcard")
df_assembler = VectorAssembler(inputCols=['time','amount','V1','V2','V3','V4','V5','V6','V7','V8','V9'
                                          ,'V10','V11','V12','V13','V14'
                                         ,'V15','V16','V17','V18','V19','V20'
                                         ,'V21','V22','V23','V24','V25','V26','V27','V28'],outputCol='features')
data= df_assembler.transform(df)
(train, test) = data.randomSplit([0.8, 0.2],seed = 11)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,Link,,erin,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'Field "V1" does not exist.'
Traceback (most recent call last):
  File "/spark/python/lib/pyspark.zip/pyspark/ml/base.py", line 173, in transform
    return self._transform(dataset)
  File "/spark/python/lib/pyspark.zip/pyspark/ml/wrapper.py", line 305, in _transform
    return DataFrame(self._java_obj.transform(dataset._jdf), dataset.sql_ctx)
  File "/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 79, in deco
    raise IllegalArgumentException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.IllegalArgumentException: 'Field "V1" does not exist.'



In [13]:
data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+-------------------+--------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-----+--------------------+-------------------+--------------------+
|                v1|                v2|                 v3|                  v4|                v5|                v6|                v7|                  v8|                 v9|               v10|               v11|                v12|                v13|                v14|                v15|                v16|                v17|                v18|               

In [14]:
train.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o275.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 111 in stage 10.0 failed 4 times, most recent failure: Lost task 111.3 in stage 10.0 (TID 492, 172.16.11.186, executor 0): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<time:double,amount:double,v1:double,v2:double,v3:double,v4:double,v5:double,v6:double,v7:double,v8:double,v9:double,v10:double,v11:double,v12:double,v13:double,v14:double,v15:double,v16:double,v17:double,v18:double,v19:double,v20:double,v21:double,v22:double,... 6 more fields>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithoutKey$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCo

## 机器学习：评估方程
- 运用BinaryClassificationEvaluator，得到每个预测结果的AUC。
- 对于二分类的数据集，得到accuracy，precision，recall，f1_score。
- 本数据集的标签为1和0，您可以通过改变等号后的数据对于其他二分类预测结果进行评估。

In [2]:
import pyspark.sql.functions as f
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="class", rawPredictionCol="prediction", metricName='areaUnderROC')
def precise_recall_f1(prediction):
    tp = prediction.filter((f.col('class')  == 1) & (f.col('prediction')  == 1)).count()
    tn = prediction.filter((f.col('class')  == 0) & (f.col('prediction')  == 0)).count()
    fp = prediction.filter((f.col('class')  == 0) & (f.col('prediction')  == 1)).count()
    fn = prediction.filter((f.col('class')  == 1) & (f.col('prediction')  == 0)).count()
    try:
        acc=float(tp+tn)/(tp+tn+fp+fn)
    except:
        acc=0
    try:
        p = float(tp)/(tp + fp)
    except:
        p = 0
    try:
        r = float(tp)/(tp + fn)
    except:
        r = 0
    try:
        f1 = 2*p*r/(p+r)
    except:
        f1 = 0
    result=[acc,p,r,f1]
    return result


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 机器学习：分类模型
- 以下分别使用了逻辑回归，决策树分类，随机森林分类，梯度提升决策树分类，线性支持向量分类。
- 将五种分类模型与数据集进行拟合并且完成预测，将预测的评估结果打印在每个单元格下。

逻辑回归（Logistic Regression）是一种用于解决二分类（0 or 1）问题的机器学习方法，用于估计某种事物的可能性。逻辑回归假设因变量 y 服从伯努利分布。

In [3]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features',labelCol = 'class').fit(train)
lrpredictions= lr.transform(test)
lrresult=['lr']
lrresult1=precise_recall_f1(lrpredictions)
lrresult=lrresult+lrresult1
lrroc=evaluator.evaluate(lrpredictions)
lrresult.append(lrroc)
print("lrresult",lrresult)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'train' is not defined
Traceback (most recent call last):
NameError: name 'train' is not defined



In [10]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol='features',labelCol = 'class')
dtmodel= dt.fit(train)
dtpredictions=dtmodel.transform(test)
dtresult=['dt']
dtresult1=precise_recall_f1(dtpredictions)
dtresult=dtresult+dtresult1
dtroc=evaluator.evaluate(dtpredictions)
dtresult.append(dtroc)
print("dtresult",dtresult)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dtresult ['dt', 0.9993835966256318, 0.9397590361445783, 0.7222222222222222, 0.8167539267015707, 0.8610669983942972]

In [11]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="Class", featuresCol="features", numTrees=10)
rfmodel= rf.fit(train)
rfpredictions=rfmodel.transform(test)
rfresult=['rf']
rfresult1=precise_recall_f1(rfpredictions)
rfresult=rfresult+rfresult1
rfroc=evaluator.evaluate(rfpredictions)
rfresult.append(rfroc)
print("rfresult",rfresult)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rfresult ['rf', 0.9993659851006499, 0.8829787234042553, 0.7685185185185185, 0.8217821782178217, 0.8841622112822686]

In [12]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="Class", featuresCol="features", maxIter=10)
gbtmodel=gbt.fit(train)
gbtpredictions=gbtmodel.transform(test)
gbtresult=['gbt']
gbtresult1=precise_recall_f1(gbtpredictions)
gbtresult=gbtresult+gbtresult1
gbtroc=evaluator.evaluate(gbtpredictions)
gbtresult.append(gbtroc)
print("gbtresult",gbtresult)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gbtresult ['gbt', 0.9993835966256318, 0.9101123595505618, 0.75, 0.8223350253807107, 0.8749294196530977]

In [13]:
from pyspark.ml.classification import LinearSVC
lsvc = LinearSVC(labelCol="Class", featuresCol="features", maxIter=10, regParam=0.1)
lsvcModel = lsvc.fit(train)
lsvcpredictions=lsvcModel.transform(test)
lsvcresult=['lsvc']
lsvcresult1=precise_recall_f1(lsvcpredictions)
lsvcresult=lsvcresult+lsvcresult1
lsvcroc=evaluator.evaluate(lsvcpredictions)
lsvcresult.append(lsvcroc)
print("lsvcresult",lsvcresult)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lsvcresult ['lsvc', 0.9992250929007943, 0.9102564102564102, 0.6574074074074074, 0.7634408602150539, 0.8286419459001642]

## 机器学习：评估表格
- 以下的表格包含了五个分类模型的五个参数

In [14]:
result=[lrresult,dtresult,rfresult,gbtresult,lsvcresult]
rdf=spark.createDataFrame(result)
rdf=rdf.toDF("", "acc", "precision", "recall","f1","auc")
rdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------+------------------+------------------+------------------+------------------+
|    |               acc|         precision|            recall|                f1|               auc|
+----+------------------+------------------+------------------+------------------+------------------+
|  lr|0.9992603159507581|            0.9125|0.6759259259259259|0.7765957446808511|0.8379012051594233|
|  dt|0.9993835966256318|0.9397590361445783|0.7222222222222222|0.8167539267015707|0.8610669983942972|
|  rf|0.9993659851006499|0.8829787234042553|0.7685185185185185|0.8217821782178217|0.8841622112822686|
| gbt|0.9993835966256318|0.9101123595505618|              0.75|0.8223350253807107|0.8749294196530977|
|lsvc|0.9992250929007943|0.9102564102564102|0.6574074074074074|0.7634408602150539|0.8286419459001642|
+----+------------------+------------------+------------------+------------------+------------------+

In [15]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
lr.write().overwrite().save("hdfs:///user/erin/data1/gbtcmodel") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…